Generating code for a model:

In [1]:
import pytorch_composer

input_dim = [4,3,32,32]
# A random sequence of neural network layers. Any positive integer shoud be a valid dimension arguement:
sequence = [
    ["Conv2d", 6],
    ["MaxPool2d", 2],
    ["Linear", 52],
    ["Relu"],
    ["MaxPool2d", 2],
    ["Linear",43],
    ["MaxPool2d", 2],
    ["Conv2d", 65],
    ["MaxPool2d", 2],
    ["Conv2d", 47],
    ["Flat"],
    ["Linear", 52],
    ["Relu"],
    ["Linear",38],
]

model_code = pytorch_composer.write_model(input_dim, sequence)
print(model_code.formatted())
# ***
# kernel_size=2, padding=0
# ***
# kernel_size=2, padding=0
# ***
# kernel_size=2, padding=0
# ***
# kernel_size=2, padding=(1, 0)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d1 = nn.Conv2d(3, 6, 3)
        self.maxpool2d1 = nn.MaxPool2d(None)
        self.linear1 = nn.Linear(15, 52)
        self.linear2 = nn.Linear(26, 43)
        self.conv2d2 = nn.Conv2d(6, 65, 3)
        self.conv2d3 = nn.Conv2d(65, 47, 3, padding=(1, 0))
        self.linear3 = nn.Linear(329, 52)
        self.linear4 = nn.Linear(52, 38)

    def forward(self, x):
        # Input dimensions : (4, 3, 32, 32)
        # Convolution layer (2d): (4, 3, 32, 32) -> (4, 6, 30, 30)
        x = self.conv2d1(x)
        # Reshaping the data: (4, 6, 30, 30) -> (4, 6, 15, 15)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 6, 15, 15) -> (4, 6, 15, 52)
        x = self.linear1(x)
        x = F.relu(x)
        # Reshaping the data: (4, 6, 15, 52) -> (4, 6, 7, 26)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 6, 7, 26) -> (4, 6, 7, 43)
        x = self.linear2(x)
        # Reshaping

Using the generated code:

In [2]:
import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


# Load and normalize the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d1 = nn.Conv2d(3, 6, kernel_size=3, padding=0)
        self.maxpool2d1 = nn.MaxPool2d(kernel_size=2, padding=0)
        self.linear1 = nn.Linear(15, 52)
        self.linear2 = nn.Linear(26, 43)
        self.conv2d2 = nn.Conv2d(6, 65, kernel_size=3, padding=0)
        self.maxpool2d2 = nn.MaxPool2d(kernel_size=2, padding=(1, 0))
        self.conv2d3 = nn.Conv2d(65, 47, kernel_size=3, padding=(1, 0))
        self.linear3 = nn.Linear(329, 52)
        self.linear4 = nn.Linear(52, 38)

    def forward(self, x):
        # Input dimensions : (4, 3, 32, 32)
        # Convolution layer (2d): (4, 3, 32, 32) -> (4, 6, 30, 30)
        x = self.conv2d1(x)
        # Reshaping the data: (4, 6, 30, 30) -> (4, 6, 15, 15)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 6, 15, 15) -> (4, 6, 15, 52)
        x = self.linear1(x)
        x = F.relu(x)
        # Reshaping the data: (4, 6, 15, 52) -> (4, 6, 7, 26)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 6, 7, 26) -> (4, 6, 7, 43)
        x = self.linear2(x)
        # Reshaping the data: (4, 6, 7, 43) -> (4, 6, 3, 21)
        x = self.maxpool2d1(x)
        # Convolution layer (2d): (4, 6, 3, 21) -> (4, 65, 1, 19)
        x = self.conv2d2(x)
        # Reshaping the data: (4, 65, 1, 19) -> (4, 65, 1, 9)
        x = self.maxpool2d2(x)
        # Convolution layer (2d): (4, 65, 1, 9) -> (4, 47, 1, 7)
        x = self.conv2d3(x)
        # Flattening the data: (4, 47, 1, 7) -> (4, 329)
        x = x.view(-1,329)
        # Linear layer: (4, 329) -> (4, 52)
        x = self.linear3(x)
        x = F.relu(x)
        # Linear layer: (4, 52) -> (4, 38)
        x = self.linear4(x)
        # Output dimensions : (4, 38)
        return x

In [4]:
# Define a Loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# Training
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        torch.save(net.state_dict(), 'model.pt')

print('Finished Training')

[1,  2000] loss: 2.468
[1,  4000] loss: 2.054
[1,  6000] loss: 1.851
[1,  8000] loss: 1.763
[1, 10000] loss: 1.645
[1, 12000] loss: 1.561
[2,  2000] loss: 1.502
[2,  4000] loss: 1.465
[2,  6000] loss: 1.418
[2,  8000] loss: 1.388
[2, 10000] loss: 1.384
[2, 12000] loss: 1.355
Finished Training
